In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv')
test = pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')

useful_features = list(train.iloc[:, 2:55].columns)


y = train.sort_values('TransactionDT')['isFraud']
X = train.sort_values('TransactionDT')[useful_features]
X_test = test[useful_features]
del train, test

X['mail_to']= X.P_emaildomain.map(str)+'-'+X.R_emaildomain.map(str)
X_test['mail_to']= X_test.P_emaildomain.map(str)+'-'+X_test.R_emaildomain.map(str)


X['card_full'] = X.card1.map(str) +'-'+X.card2.map(str)+ '-' +X.card3.map(str)+'-'+X.card5.map(str)
X_test['card_full'] = X_test.card1.map(str) +'-'+X_test.card2.map(str)+ '-' +X_test.card3.map(str)+'-'+X_test.card5.map(str)

X['Transaction_day_of_week'] = np.floor((X['TransactionDT'] / (3600 * 24) - 1) % 7)
X_test['Transaction_day_of_week'] = np.floor((X_test['TransactionDT'] / (3600 * 24) - 1) % 7)
X['Transaction_hour'] = np.floor(X['TransactionDT'] / 3600) % 24
X_test['Transaction_hour'] = np.floor(X_test['TransactionDT'] / 3600) % 24

X['P_isproton']=(X['P_emaildomain']=='protonmail.com')
X['R_isproton']=(X['R_emaildomain']=='protonmail.com')
X_test['P_isproton']=(X_test['P_emaildomain']=='protonmail.com')
X_test['R_isproton']=(X_test['R_emaildomain']=='protonmail.com')

X['TransactionAmt_decimal'] = ((X['TransactionAmt'] - X['TransactionAmt'].astype(int)) * 1000).astype(int)
X_test['TransactionAmt_decimal'] = ((X_test['TransactionAmt'] - X_test['TransactionAmt'].astype(int)) * 1000).astype(int)

X.drop(['TransactionDT'],axis=1, inplace=True)
X_test.drop(['TransactionDT'],axis=1, inplace=True)


categorical_features = [
    'ProductCD',
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
    'addr1', 'addr2',
    'P_emaildomain',
    'R_emaildomain',
    'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9','Transaction_day_of_week','Transaction_hour','mail_to','card_full'
]




continuous_features = list(filter(lambda x: x not in categorical_features, X))



In [ ]:
class ContinuousFeatureConverter:
    def __init__(self, name, feature, log_transform):
        self.name = name
        self.skew = feature.skew()
        self.log_transform = log_transform
        
    def transform(self, feature):
        if self.skew > 1:
            feature = self.log_transform(feature)
        
        mean = feature.mean()
        std = feature.std()
        return (feature - mean)/(std + 1e-6)        
    
from tqdm.autonotebook import tqdm

feature_converters = {}
continuous_features_processed = []
continuous_features_processed_test = []

for f in tqdm(continuous_features):
    feature = X[f]
    feature_test = X_test[f]
    log = lambda x: np.log10(x + 1 - min(0, x.min()))
    converter = ContinuousFeatureConverter(f, feature, log)
    feature_converters[f] = converter
    continuous_features_processed.append(converter.transform(feature))
    continuous_features_processed_test.append(converter.transform(feature_test))
    
continuous_train = pd.DataFrame({s.name: s for s in continuous_features_processed}).astype(np.float64)
continuous_test = pd.DataFrame({s.name: s for s in continuous_features_processed_test}).astype(np.float64)


continuous_train['isna_sum'] = continuous_train.isna().sum(axis=1)
continuous_test['isna_sum'] = continuous_test.isna().sum(axis=1)

continuous_train['isna_sum'] = (continuous_train['isna_sum'] - continuous_train['isna_sum'].mean())/continuous_train['isna_sum'].std()
continuous_test['isna_sum'] = (continuous_test['isna_sum'] - continuous_test['isna_sum'].mean())/continuous_test['isna_sum'].std()

isna_columns = []


for column in tqdm(continuous_features):
    isna = continuous_train[column].isna()
    if isna.mean() > 0.:
        continuous_train[column + '_isna'] = isna.astype(int)
        continuous_test[column + '_isna'] = continuous_test[column].isna().astype(int)
        isna_columns.append(column)
        
continuous_train = continuous_train.fillna(0.)
continuous_test = continuous_test.fillna(0.)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm.autonotebook import tqdm
#from category_encoders import one_hot


def categorical_encode(df_train, df_test, categorical_features, n_values=50):
    df_train = df_train[categorical_features].astype(str)
    df_test = df_test[categorical_features].astype(str)
    
    categories = []
    for column in tqdm(categorical_features):
        categories.append(list(df_train[column].value_counts().iloc[: n_values - 1].index) + ['Other'])
        values2use = categories[-1]
        df_train[column] = df_train[column].apply(lambda x: x if x in values2use else 'Other')
        df_test[column] = df_test[column].apply(lambda x: x if x in values2use else 'Other')
        
    
    ohe = OneHotEncoder(categories = categories)
    ohe.fit(pd.concat([df_train, df_test]))
    df_train = pd.DataFrame(ohe.transform(df_train).toarray()).astype(np.int8)
    df_test = pd.DataFrame(ohe.transform(df_test).toarray()).astype(np.int8)
    return df_train, df_test


train_categorical, test_categorical = categorical_encode(X, X_test, categorical_features)

X = pd.concat([continuous_train, train_categorical], axis=1)
#del continuous_train, train_categorical
X_test = pd.concat([continuous_test, test_categorical], axis=1)
#del continuous_test, test_categorical

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

clf = LinearDiscriminantAnalysis(solver = 'svd')

test = pd.DataFrame()

test['X'] = clf.fit_transform(X.iloc[:,2:36],y).flatten()

X['lda271'] = test.X

X_test['X'] = clf.transform(X_test.iloc[:,2:36]).flatten()


In [ ]:

import tensorflow as tf 
import keras
from keras import layers
from keras import losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, accuracy_score,roc_auc_score
import matplotlib.pyplot as plt
from keras.models import Model

In [ ]:
from sklearn.model_selection import KFold



models = list()

for i in range (1,21):
    models.append("model"+str(i))

def get_model():
    modelz = keras.Sequential([
    
    keras.layers.Dense(256, activation='relu',kernel_initializer=tf.initializers.he_uniform(),input_shape=(642,)),
    #keras.layers.BatchNormalization(),
    keras.layers.Dense(64,activation='relu',kernel_initializer=tf.initializers.he_uniform()),
    keras.layers.Dense(16,activation='relu',kernel_initializer=tf.initializers.he_uniform()),
    keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) ])
    modelz.compile(optimizer='Adam',
              loss='binary_crossentropy' ,metrics = ['accuracy'])
    
    return modelz
    

def get_name(pos):
    return models[pos]



import gc
#gc.collect()


In [ ]:
split_ind = int(X.shape[0]*0.80)

X_tr = X.iloc[:split_ind]
X_val = X.iloc[split_ind:]

y_tr = y.iloc[:split_ind]
y_val = y.iloc[split_ind:]

In [ ]:
kf = KFold(n_splits=20)
from sklearn.model_selection import ShuffleSplit
kf.get_n_splits(X_tr,y_tr)
n=0

ss = ShuffleSplit(n_splits  =  20 ,test_size = 0.6, random_state = 47)
ss.get_n_splits(X_tr,y_tr)
for train_index, test_index in kf.split(X_tr,y_tr):
    
    train = X_tr.iloc[train_index]
    labels_traini = y_tr.iloc[train_index]
    
    test_y=y_tr.iloc[test_index]
    test=X_tr.iloc[test_index]
    
    models[n]=get_model()
    models[n].fit(train,labels_traini,epochs = 8 ,batch_size = 2048,verbose = 2)
    
    n=n+1
    print(n,'model training')
 #print(roc_auc_score(labels_traini,models[n].predict_proba(train)))  

In [ ]:
#cyka = pd.DataFrame(models[2].predict_proba(X_tr).astype('float32').flatten())

predictions = pd.DataFrame()
for n in range(0,20):
    print('predicting',n,'th model')
    predictions[str(n)] = models[n].predict_proba(X_tr).astype('float64').flatten()

In [ ]:
predictions_val = pd.DataFrame()
for n in range(0,20):
    print('predicting',n,'th model')
    predictions_val[str(n)] = models[n].predict_proba(X_val).astype('float64').flatten()


In [ ]:
modelx = keras.Sequential([
    keras.layers.Dense(64, activation= 'relu',kernel_initializer=tf.initializers.he_uniform(),input_shape=(20,)),
    keras.layers.Dropout(0.3),
    #keras.layers.Dense(20, activation='relu',kernel_initializer=tf.initializers.he_uniform()),
    keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) ])



modelx.compile(optimizer=keras.optimizers.Adam(0.01),
              loss='binary_crossentropy')



modelx.fit(predictions,y_tr,epochs = 3,batch_size = 2048,class_weight = {0:1,1:1} ,verbose = 2)


modelx.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy')


modelx.fit(predictions,y_tr,epochs = 1,batch_size = 2048,class_weight = {0:1,1:1} ,verbose = 2)


modelx.compile(optimizer=keras.optimizers.Adam(0.0001),
              loss='binary_crossentropy')
modelx.fit(predictions,y_tr,epochs = 1,batch_size = 10000,class_weight = {0:1,1:1} ,verbose = 2)

pred = modelx.predict_proba(predictions)
pred_2 = modelx.predict_proba(predictions_val)
    
#validation_data=(data_test,label_test)

print('F1_score:',f1_score(y_tr , np.round(pred)),'------>',f1_score(y_val , np.round(pred_2)))
print('Confusion matrix :',confusion_matrix(y_tr,np.round(pred)),'------>\n',confusion_matrix(y_val,np.round( pred_2)))
print('Roc Score' ,roc_auc_score(y_tr, pred),'------>',roc_auc_score(y_val, pred_2))


#model.fit(predictions_val,y_val,epochs = 3,batch_size = 512,class_weight = {0:1,1:1} ,verbose = 2)

In [ ]:
predictions_test = pd.DataFrame()
for n in range(0,20):
    print('predicting',n,'th model')
    predictions_test[str(n)] = models[n].predict_proba(X_test).astype('float64').flatten()

In [ ]:
sub = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')


sub.isFraud = pd.DataFrame(modelx.predict_proba(predictions_test).flatten())




sub.to_csv('sub_bagging.csv',header =True,index = False)